In [4]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws_seg2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [3]:
sphynx.stop()

In [6]:
# 각 day 28
start_date = "2022-06-08"
end_date = "2022-07-05"

ws_start_date = "2022-07-13"
ws_day_28 = "2022-08-09"

In [7]:
gcoin = load_data_mart("pc", ws_start_date, ws_day_28, "gcoin_use")
wsus_gcoin = gcoin.where(col("event_name").like("%wsus%"))
wsus_user = wsus_gcoin.select("account_id").distinct()
costume = gcoin.where((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                    ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%survivorpass%"))) | \
                    (col("event_name").like("%workshop%"))
                     )
costume_user = costume.select("account_id").distinct()

In [8]:
only_costume_gcoin = costume.join(wsus_user, "account_id", "leftanti")
only_wsus_gcoin= wsus_gcoin.join(costume_user, "account_id", "leftanti")

In [9]:
costume_and_wsus_gcoin_user = wsus_gcoin.join(costume_user, "account_id").select("account_id").distinct()

In [10]:
costume_and_wsus_gcoin = costume.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price") \
    .unionByName(wsus_gcoin.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price")) \
    .join(costume_and_wsus_gcoin_user, "account_id")

In [35]:
only_costume_gcoin.where(col("event_name").like("%yourshop%")).select(countDistinct("account_id").alias("user_cnt")).show()

+--------+
|user_cnt|
+--------+
|  321319|
+--------+



In [13]:
only_costume_gcoin.withColumn("yourshop_gcoin", when((~col("event_name").like("%yourshop%")) & (~col("event_name").like("%workshop%")), col("paid_use") + col("free_use")).otherwise(lit(0))) \
    .withColumn("workshop_gcoin", when(col("event_name").like("%workshop%"), col("paid_use") + col("free_use")).otherwise(lit(0))) \
    .groupBy("account_id").agg(sum("yourshop_gcoin").alias("yourshop"), sum("workshop_gcoin").alias("workshop")).where(col("workshop") == 0).select(countDistinct('account_id')).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    369066|
+--------------------------+



In [14]:
costume_and_wsus_gcoin.withColumn("yourshop_gcoin", when((~col("event_name").like("%yourshop%")) & (~col("event_name").like("%workshop%"))& (~col("event_name").like("%wsus%")), col("paid_use") + col("free_use")).otherwise(lit(0))) \
    .withColumn("workshop_gcoin", when(col("event_name").like("%workshop%"), col("paid_use") + col("free_use")).otherwise(lit(0))) \
    .groupBy("account_id").agg(sum("yourshop_gcoin").alias("yourshop"), sum("workshop_gcoin").alias("workshop")).where(col("workshop") == 0).select(countDistinct('account_id')).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     53981|
+--------------------------+



## a

In [9]:
from pyspark.sql.types import * 

def chest_cnt(product_id):
    if product_id in ["itemdesc.14300001", "itemdesc.14300002", "itemdesc.13000577", "itemdesc.13000580"]:
        return 1
    elif product_id in ["itemdesc.13000578", "itemdesc.13000581"]:
        return 5
    elif product_id == "itemdesc.13000582":
        return 10
    else:
        return 0

assign_chest_cnt = udf(chest_cnt, IntegerType())
only_costume_gcoin = only_costume_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [10]:
only_costume_gcoin_by_user = only_costume_gcoin.groupBy("account_id") \
    .agg(sum(col("free_use") + col("paid_use")).alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt"))

In [11]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("group", \
                                         when(col("total_gcoin") <= 1000, "A") \
                                        .when((col("total_gcoin") > 1000) & (col("total_gcoin") <= 5000), "B") \
                                        .when((col("total_gcoin") > 5000) & (col("total_gcoin") <= 10000), "C") \
                                        .when((col("total_gcoin") > 10000) & (col("total_gcoin") <= 30000), "D") \
                                        .when((col("total_gcoin") > 30000), "E") \
                                     )
only_costume_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_gcoin").alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_gcoin,paid_use,chest_cnt
0,A,615905,241758400,40659430,388910
1,B,249039,458716520,192198270,880056
2,C,25193,174792780,144035990,582439
3,D,22689,382987990,335592650,1256226
4,E,9032,543596750,485418955,1391368


In [12]:
only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,chest_cnt
0,A,costume,302273,137240160,23985880,0
1,A,workshop,351998,104518240,16673550,388910
2,B,costume,165675,225969070,78040935,0
3,B,workshop,144567,232747450,114157335,880056
4,C,costume,11118,19742510,15891260,0
5,C,workshop,24541,155050270,128144730,582439
6,D,costume,9563,15107100,12541640,0
7,D,workshop,22654,367880890,323051010,1256226
8,E,costume,4270,8093770,6967200,0
9,E,workshop,9032,535502980,478451755,1391368


In [13]:
only_costume_gcoin_by_user = only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("chest_cnt").alias("chest_cnt")).toPandas()
only_costume_gcoin_by_user.to_csv("./only_costume_gcoin_by_user2_day28.csv", index=False)

## b

In [14]:
def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [15]:
assign_chest_cnt = udf(chest_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [16]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [17]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("group", \
                                                                when(col("wsus_cnt") <=3, "A") \
                                                                .when((col("wsus_cnt") >= 4) & (col("wsus_cnt") <= 15), "B") \
                                                                .when((col("wsus_cnt") >= 16) & (col("wsus_cnt") <= 30), "C") \
                                                                .when((col("wsus_cnt") >= 31) & (col("wsus_cnt") <= 240), "D") \
                                                                .when((col("wsus_cnt") >= 241), "E") \
                                                                          )
costume_and_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,97949,223689790,138101920,154754,502222
1,B,42686,273959360,205827980,347381,586215
2,C,13077,197290180,169516070,299102,402942
3,D,21990,747432550,660675265,1848113,1091873
4,E,2801,336525210,300876250,1304408,230220


In [18]:
costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,costume,51629,44002710,15928825,5677,0
1,A,workshop,62765,149871680,117498680,0,502222
2,A,wsus,97949,29815400,4674415,149077,0
3,B,costume,23750,25806380,16716040,3058,0
4,B,workshop,29785,182962180,157543660,0,586215
5,B,wsus,42686,65190800,31568280,344323,0
6,C,costume,7266,10998720,8828720,627,0
7,C,workshop,10657,131534460,115589845,0,402942
8,C,wsus,13077,54757000,45097505,298475,0
9,D,costume,13059,26467170,22105925,1283,0


In [19]:
costume_and_wsus_gcoin_by_user2 = costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).toPandas()
costume_and_wsus_gcoin_by_user2.to_csv("./costume_and_wsus_gcoin_by_user2_day28.csv", index=False)

## c

In [20]:
only_wsus_gcoin = only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [21]:
only_wsus_gcoin_by_user = only_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [22]:
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("group", \
                                                when(col("wsus_cnt") <=10, "A") \
                                                .when((col("wsus_cnt") >= 11) & (col("wsus_cnt") <= 50), "B") \
                                                .when((col("wsus_cnt") >= 51) & (col("wsus_cnt") <= 100), "C") \
                                                .when((col("wsus_cnt") >= 101) & (col("wsus_cnt") <= 800), "D") \
                                                .when((col("wsus_cnt") >= 801), "E") \
                                                          )
only_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,240547,159662000,8546630,832796
1,B,11018,42561800,30280740,231581
2,C,1839,23142000,20118225,127638
3,D,1231,43278400,38417535,239473
4,E,27,5156800,4564370,28629


In [23]:
only_wsus_gcoin_by_user.toPandas().to_csv("./only_wsus_gcoin_by_user2_day28.csv", index=False)